In [1]:
#r "nuget: System.Threading.Channels"
using System.Collections.Concurrent;
using System.Linq;
using System.Threading.Channels;
using System.Threading.Tasks;

Installed package System.Threading.Channels version 5.0.0

### Bounded Channels are limited in how many message they can hold

In [2]:
var channel = Channel.CreateBounded<int>(6);

In [3]:
Parallel.ForEach(Enumerable.Range(0, 4), n1 =>
{
    var couldNotBeAdded = new ConcurrentQueue<int>();

    Parallel.ForEach(Enumerable.Range(0, 4), n2 =>
    {
        if (!channel.Writer.TryWrite(n1 * 4 + n2))
        {
            couldNotBeAdded.Enqueue(n1 * 4 + n2);
        }
    });

    Console.WriteLine($"Unsent messages [{n1}]: {string.Join(",", couldNotBeAdded)}");
});

Task.Delay(1000).ContinueWith(_ => channel.Writer.Complete());

Unsent messages [1]: 6,5
Unsent messages [3]: 14,13
Unsent messages [2]: 10,11,9,8
Unsent messages [0]: 2,1


In [4]:
Console.WriteLine("Received messages:");
await foreach (var message in channel.Reader.ReadAllAsync())
{
    Console.WriteLine(message);
}

Received messages:
0
4
3
7
12
15


### Unbounded Channels don't have these limits

In [6]:
// Try with Unbounded versus Bounded
var channel = Channel.CreateUnbounded<int>();

In [7]:
Parallel.ForEach(Enumerable.Range(0, 4), n1 =>
{
    var couldNotBeAdded = new ConcurrentQueue<int>();

    Parallel.ForEach(Enumerable.Range(0, 4), n2 =>
    {
        if (!channel.Writer.TryWrite(n1 * 4 + n2))
        {
            couldNotBeAdded.Enqueue(n1 * 4 + n2);
        }
    });

    Console.WriteLine($"Unsent messages [{n1}]: {string.Join(",", couldNotBeAdded)}");
});

Task.Delay(1000).ContinueWith(_ => channel.Writer.Complete());

Unsent messages [3]: 
Unsent messages [0]: 
Unsent messages [2]: 
Unsent messages [1]: 


In [8]:
Console.WriteLine("Received messages:");
await foreach (var message in channel.Reader.ReadAllAsync())
{
    Console.WriteLine(message);
}

Received messages:
12
8
0
4
5
13
9
14
15
1
2
3
6
10
11
7
